In [1]:
import numpy as np
import os
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

2024-07-05 00:15:21.289200: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-05 00:15:21.289352: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-05 00:15:21.292850: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-05 00:15:21.310772: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-05 00:15:23.462783: W tensorflow/compiler/tf2

In [2]:
# Path to your CASIA2 dataset
PATH = '../data/CASIA2'

# Directories for authentic and tampered images
authentic_dir = os.path.join(PATH, 'Au')
tampered_dir = os.path.join(PATH, 'Tp2')

authentic_number = 200
tampered_number = 200
IMG_SIZE = (128, 128)

def load_images_from_directory(directory_path, n, i=0):
    images = []
    for filename in os.listdir(directory_path)[i:n]:
        if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
            image_path = os.path.join(directory_path, filename)
            imagen_original = tf.io.read_file(image_path)
            imagen_original = tf.image.decode_jpeg(imagen_original, channels=3)
            resized = tf.image.resize(imagen_original, IMG_SIZE)
            images.append(resized)
    return images

# Get file lists and labels
authentic_files = load_images_from_directory(authentic_dir, authentic_number)
tampered_files = load_images_from_directory(tampered_dir, tampered_number)
authentic_labels = [0] * len(authentic_files)
tampered_labels = [1] * len(tampered_files)

# Combine authentic and tampered data
all_files = authentic_files + tampered_files
all_labels = authentic_labels + tampered_labels

In [3]:
all_files = np.array(all_files)
all_labels = np.array(all_labels)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    all_files, all_labels, test_size=0.4, random_state=42
)

y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=2)

In [4]:
def unet_model(input_size=(128, 128, 3), filters=32):
    inputs = layers.Input(input_size)
    
    # Encoder (Downsampling)
    def encoder_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
        c = layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
        c = layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
        p = layers.MaxPooling2D((2, 2))(c)
        return c, p

    # Decoder (Upsampling)
    def decoder_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
        u = layers.UpSampling2D((2, 2))(x)
        u = layers.concatenate([u, skip])
        c = layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(u)
        c = layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
        return c

    # Encoder
    c1, p1 = encoder_block(inputs, filters)
    c2, p2 = encoder_block(p1, filters*2)
    c3, p3 = encoder_block(p2, filters*4)
    c4, p4 = encoder_block(p3, filters*8)

    # Bridge
    b = layers.Conv2D(filters*16, (3, 3), padding="same", activation="relu")(p4)
    b = layers.Conv2D(filters*16, (3, 3), padding="same", activation="relu")(b)
    b = layers.Dropout(0.3)(b)

    # Decoder
    d1 = decoder_block(b, c4, filters*8)
    d2 = decoder_block(d1, c3, filters*4)
    d3 = decoder_block(d2, c2, filters*2)
    d4 = decoder_block(d3, c1, filters)

    # Global Average Pooling
    gap = layers.GlobalAveragePooling2D()(d4)

    # Dense layers for classification
    dense1 = layers.Dense(64, activation='relu')(gap)
    dense2 = layers.Dense(32, activation='relu')(dense1)
    outputs = layers.Dense(2, activation='softmax')(dense2)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    
    return model

In [5]:
model = unet_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 32)         896       ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 128, 128, 32)         9248      ['conv2d[0][0]']              
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 64, 64, 32)           0         ['conv2d_1[0][0]']            
 D)                                                                                           

In [6]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)

Epoch 1/10
8/8 [==============================] - 70s 8s/step - loss: 1.3713 - accuracy: 0.5208 - val_loss: 0.7558 - val_accuracy: 0.5250
Epoch 2/10
8/8 [==============================] - 65s 8s/step - loss: 0.7223 - accuracy: 0.5458 - val_loss: 0.7540 - val_accuracy: 0.4688
Epoch 3/10
8/8 [==============================] - 68s 9s/step - loss: 0.7052 - accuracy: 0.5292 - val_loss: 0.6921 - val_accuracy: 0.5312
Epoch 4/10
5/8 [=================>............] - ETA: 19s - loss: 0.6862 - accuracy: 0.5500

In [ ]:
model.summary()

In [ ]:
tests_number = 100

tampered_test = load_images_from_directory(authentic_dir, tests_number + tampered_number, i = tampered_number)
untampered_test = load_images_from_directory(tampered_dir, tests_number + authentic_number, i = authentic_number)
tampered_labels = [0] * len(tampered_test)
untampered_labels = [1] * len(untampered_test)

X_test = np.array(tampered_test + untampered_test)
y_test = np.array(tampered_labels + untampered_labels)

# Crear un Dataset a partir de los datos
dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Función para preprocesar las imágenes si es necesario
def preprocess_image(image, label):
    # Aquí puedes realizar cualquier preprocesamiento adicional necesario
    return image, label

# Aplicar el preprocesamiento al Dataset
dataset = dataset.map(preprocess_image)

# Definir el tamaño del lote y mezclar el Dataset (opcional)
batch_size = 32
dataset = dataset.batch(batch_size)

# Crear un iterador sobre el Dataset
iterator = iter(dataset)

# Lista para almacenar las predicciones y etiquetas reales
predictions = []

# Iterar sobre el Dataset para obtener predicciones
for batch in dataset:
    batch_images, batch_labels = batch
    
    # Obtener predicciones para el lote actual
    batch_predictions = model.predict(batch_images)
    
    # Iterar sobre las predicciones del lote y guardarlas junto con las etiquetas reales
    for i in range(len(batch_predictions)):
        input_image = batch_images[i]
        true_label = batch_labels[i]
        predicted_label = 1 if batch_predictions[i][0] > batch_predictions[i][1] else 0
        
        predictions.append((input_image, true_label, predicted_label))

# Evaluar el rendimiento comparando las predicciones con las etiquetas reales
correct_predictions = 0
total_images = len(predictions)

for input_image, true_label, predicted_label in predictions:
    if true_label == predicted_label:
        correct_predictions += 1

accuracy = correct_predictions / total_images
print(f"Accuracy: {accuracy:.2f}")

# Opcional: Imprimir algunas predicciones para verificar
for input_image, true_label, predicted_label in predictions[:100]:  # Imprimir las primeras 10 predicciones
    print(f"True Label: {true_label}, Predicted Label: {predicted_label}")